# 10. Data acquisition


### Exercise 10.1.

The list below contains a number of URLs. They are the web addresses of texts created for the [Project Gutenberg](https://www.gutenberg.org) website.

```
urls = [ 'https://www.gutenberg.org/files/580/580-0.txt' ,
'https://www.gutenberg.org/files/1400/1400-0.txt' ,
'https://www.gutenberg.org/files/786/786-0.txt' ,
'https://www.gutenberg.org/files/766/766-0.txt' 
]
```

Write a program in Python that can download all the files in this list and stores them in the current directory.

As filenames, use the same names that are used by Project Gutenberg (e.g. '580-0.txt' or '1400-0.txt').

The basename in a URL can be extracted using the [`os.path.basename()`](https://docs.python.org/3/library/os.path.html#os.path.basename) function.


In [1]:
import requests
import os.path

# Recreate the given list using copy and paste
urls = [ 'https://www.gutenberg.org/files/580/580-0.txt' ,
'https://www.gutenberg.org/files/1400/1400-0.txt' ,
'https://www.gutenberg.org/files/786/786-0.txt' ,
'https://www.gutenberg.org/files/766/766-0.txt' 
]

# We use a for-loop to take the same steps for each item in the list:
for url in urls:
    # 1. Download the file contents
    response = requests.get(url)
    # 1a. Force the textual contents to be interpreted as UTF-8 encoded, because the website does not send the text encoding
    response.encoding = 'utf-8'
    # 2. Use basename to get a suitable filename
    filename = os.path.basename(url)
    # 3. Open the file in write mode and write the downloaded file contents to the file
    out = open( filename , mode = 'w', encoding= 'utf-8' )
    out.write( response.text )
    # 4. Close the file
    out.close()
    
print('Done!')
    

Done!


### Exercise 10.2.

As was discussed, you can use the *MusicBrainz* API to request information about musicians. Via the code that is provided, you can request the names and the types of artists. This specific API can make much more information available, however. Try to add some code with can add the following data about each artist: 

* The gender (in the case of a person)
* The date of birth (in the case of a person) or formation (in the case of a group)
* Aliases

If you want to see the structure of the JSON data, you can 'uncomment' the print statement in the second cell to be able explore the structure of the JSON data. 

The information about the date of birth or the date of formation is available via the key `life-span`. The value associated with this key is yet another dictionary. This second dictionary has the keys you need, namely `start` and `end`.

In [2]:
import requests
from requests.utils import requote_uri


root_url = 'https://musicbrainz.org/ws/2/'

## The parameters for the API call are defined as variables
entity = 'artist'
query = 'David Bowie'
limit = 5
fmt = 'json'

query = requote_uri(query)

api_call = f'{root_url}{entity}?query={query}&fmt={fmt}&limit={limit}'
response = requests.get( api_call )

In [3]:
import json

musicbrainz_results = response.json()

for artist in musicbrainz_results['artists']:
    #print(json.dumps(artist, indent=4))
    name = artist.get('name','[unknown]')
    artist_type = artist.get('type','[unknown]')
    print(f'{name} ({artist_type})')
    
    
    if artist_type == 'Person':
        if 'gender' in artist:
            print(f'Gender: {artist["gender"].title()}')

    begin = ''
    
    if 'life-span' in artist:
        begin = artist['life-span'].get('begin','[unknown]')

        if artist_type == 'Person':
            print('Born',end=': ')
        else:
            print('Formation',end=': ')
        print(begin)
    
    
    aliases = []
    if 'aliases' in artist:
        for alias in artist['aliases']:
            aliases.append(alias['sort-name'])
        
    if len(aliases)>0:
        print('Aliases',end=': ')
        print( ', '.join(aliases) )
        
    print('\n')
        

David Bowie (Person)
Gender: Male
Born: 1947-01-08
Aliases: 데이비드 보위, Ziggy Stardust, David Jones, Jones, David Robert, Дэвид Боуи, The Thin White Duke, デビッド・ボウイ, Davis Bowie, Bowie, Davie Bowie, Jones, Davy, デヴィッド・ボウイ


David Bowie Band (Group)
Formation: [unknown]


Celebrating David Bowie (Group)
Formation: 2017-01-08


The David Bowie Knives (Group)
Formation: [unknown]


David Bowie & The Buzz (Group)
Formation: 1966-02-10




### Exercise 10.3.

Find the coordinates for each address in the given list using [OpenStreetMap](https://www.openstreetmap.org/)'s Nominatim API.

The Nominatim API can be used, among other things, to find the precise geographic coordinates of a specific location. The base URL of this API is <https://nominatim.openstreetmap.org/search>.

Following the `q` parameter, you need to supply a string describing the locations whose latitude and longitude you want to find. As values for the `format` parameter, you can use `xml` for XML-formatted data or `json` for JSON-formatted data. 

Use this API to find the longitude and the latitude of the addresses in the following list:

```
addresses = ['Grote Looiersstraat 17 Maastricht' , 'Witte Singel 27 Leiden' ,
'Singel 425 Amsterdam' , 'Drift 27 Utrecht' , 'Broerstraat 4 Groningen']
```

The JSON data received via the OpenStreetMap API can be converted to regular Python lists and dictionaries using the `json()` method: 

```json_data = response.json()```

If the result is saved as variable named `json_data`, you should be able to access the latitude and the longitude as follows:

```
latitude = json_data[0]['lat']
longitude = json_data[0]['lon']
```

The `[0]` is used to get the results for the first result.

Print each address and its latitude and longitude coordinates.

In [4]:
import requests

addresses = ['Grote Looiersstraat 17 Maastricht' , 'Witte Singel 27 Leiden' ,
'Singel 425 Amsterdam' , 'Drift 27 Utrecht' , 'Broerstraat 4 Groningen']


for a in addresses:
    url = f'https://nominatim.openstreetmap.org/search?q={a}&format=json'

    response = requests.get( url ) # The spaces in each address are automatically encoded as '%20' by requests
    json_data = response.json()
    # json_data is a list of results; we assume that the first result is always correct(!)
    latitude = json_data[0]['lat']
    longitude = json_data[0]['lon']
    print( f'{latitude},{longitude}')


50.8450593,5.6899146
52.1572514,4.4815708
52.3676855,4.8897127
52.0946054,5.1235846
53.2189163,6.5629461


### Exercise 10.4.

*[PLOS One](https://journals.plos.org/plosone/)* is a peer reviewed open access journal. The *PLOS One* API can be used to request metadata about all the articles that have been published in the journal. In this API, you can refer to specific articles using their [DOI](https://www.doi.org/).

Such requests can be sent using API calls with the following structure:

https://api.plos.org/search?q=id:{doi}

To acquire data about the article with DOI [10.1371/journal.pone.0270739](https://doi.org/10.1371/journal.pone.0270739), for example, you can use the following API call:

https://api.plos.org/search?q=id:10.1371/journal.pone.0270739

Try to write code which can get hold of metadata about the articles with the following DOIs:

* 10.1371/journal.pone.0169045
* 10.1371/journal.pone.0271074
* 10.1371/journal.pone.0268993

For each article, print the title, the publication date, the article type, a list of all the authors and the abstract. 


In [5]:
import requests

dois = [ '10.1371/journal.pone.0169045','10.1371/journal.pone.0268993','10.1371/journal.pone.0271074' ]


root_url = 'https://api.plos.org/search?q=id:'

## The parameters for the API call are defined as variables
for doi in dois:

    api_call = f'{root_url}{doi}'
    print(api_call)

    response = requests.get( api_call )

    if response: 
        plos_results = response.json()
        for article in plos_results['response']['docs']:
            #print(article)
            print(article['title_display'])
            print(article['article_type'])
            print(article['publication_date'])
            authors = article['author_display']
            for author in authors:
                print(author)
            print(article['abstract'][0].strip())
            print('\n')



https://api.plos.org/search?q=id:10.1371/journal.pone.0169045
Quasistatic Cavity Resonance for Ubiquitous Wireless Power Transfer
Research Article
2017-02-15T00:00:00Z
Matthew J. Chabalko
Mohsen Shahmohammadi
Alanson P. Sample
Wireless power delivery has the potential to seamlessly power our electrical devices as easily as data is transmitted through the air. However, existing solutions are limited to near contact distances and do not provide the geometric freedom to enable automatic and un-aided charging. We introduce quasistatic cavity resonance (QSCR), which can enable purpose-built structures, such as cabinets, rooms, and warehouses, to generate quasistatic magnetic fields that safely deliver kilowatts of power to mobile receivers contained nearly anywhere within. A theoretical model of a quasistatic cavity resonator is derived, and field distributions along with power transfer efficiency are validated against measured results. An experimental demonstration shows that a 54 m3 QSCR 

### Exercise 10.5.

This tutorial has explained how you can extract data about the titles and the prices of all the books that are shown on the web page [http://books.toscrape.com/](http://books.toscrape.com/). 

Can you write code to extract the URLs of all the book covers on this page? These URLs can be found in the `src` attribute of the `<img>` elements within the `<article>` about each book. Note that the `<img>` element specifies a relative path. To change the relative path into an absolute path, you need to concatenate the base url ([http://books.toscrape.com/](http://books.toscrape.com/) and the relative path to the image. 

In [6]:
from bs4 import BeautifulSoup
import requests

url = 'http://books.toscrape.com/'
response = requests.get( url )


if response:
    response.encoding = 'utf-8'
    html_page = response.text 
    

soup = BeautifulSoup(html_page, "lxml")

all_books = soup.find_all( 'article' , {'class':'product_pod'} )
    
for book in all_books:
    i = book.find('img')
    image_url = url + i.get('src')
    print(image_url)
    

http://books.toscrape.com/media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg
http://books.toscrape.com/media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg
http://books.toscrape.com/media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg
http://books.toscrape.com/media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg
http://books.toscrape.com/media/cache/be/a5/bea5697f2534a2f86a3ef27b5a8c12a6.jpg
http://books.toscrape.com/media/cache/68/33/68339b4c9bc034267e1da611ab3b34f8.jpg
http://books.toscrape.com/media/cache/92/27/92274a95b7c251fea59a2b8a78275ab4.jpg
http://books.toscrape.com/media/cache/3d/54/3d54940e57e662c4dd1f3ff00c78cc64.jpg
http://books.toscrape.com/media/cache/66/88/66883b91f6804b2323c8369331cb7dd1.jpg
http://books.toscrape.com/media/cache/58/46/5846057e28022268153beff6d352b06c.jpg
http://books.toscrape.com/media/cache/be/f4/bef44da28c98f905a3ebec0b87be8530.jpg
http://books.toscrape.com/media/cache/10/48/1048f63d3b5061cd2f424d20b3f9b666.jpg
http://books.toscrape.com/me

### Exercise 10.6. 

On the web page [http://books.toscrape.com/](http://books.toscrape.com/), the menu on the lefthand side contains a list of all the subject categories of the books. 

Try to write some code which can extract all the terms in this list. This list is in an element named `div`, and this `<div>` has a `class` attribute with the value `side_categories`. The categories themselves are all encoded within an `<a>` element. 

In [7]:
from bs4 import BeautifulSoup
import requests

url = 'http://books.toscrape.com/'
response = requests.get( url )


if response:
    response.encoding = 'utf-8'
    html_page = response.text 
    
soup = BeautifulSoup(html_page, "lxml")

div = soup.find( 'div' , {'class':'side_categories'} )

all_categories = div.find_all('a')
    
for category in all_categories:
    print(category.text.strip())
    

Books
Travel
Mystery
Historical Fiction
Sequential Art
Classics
Philosophy
Romance
Womens Fiction
Fiction
Childrens
Religion
Nonfiction
Music
Default
Science Fiction
Sports and Games
Add a comment
Fantasy
New Adult
Young Adult
Science
Poetry
Paranormal
Art
Psychology
Autobiography
Parenting
Adult Fiction
Humor
Horror
History
Food and Drink
Christian Fiction
Business
Biography
Thriller
Contemporary
Spirituality
Academic
Self Help
Historical
Christian
Suspense
Short Stories
Novels
Health
Politics
Cultural
Erotica
Crime


### Exercise 10.7.

Write a program in Python which can extract data from the following web page:

[https://www.imdb.com/chart/top/](https://www.imdb.com/chart/top/)

This is a page on the [Internet Movie Database website](https://www.imdb.com). It lists the 25 most highly rated movies.

More specifically, try to extract the titles of these movies and the URLs of the pages on IMDB. 

If you inspect the source code of this web page, you can see that the information about the movies is encoded as follows:

```
<td class="titleColumn">

<a href="/title/tt0068646/">
The Godfather
</a>

</td>
```

The data can found in a `<td>` element whose `class` attribute has value `titleColumn`. `td` stands for 'table data'. This HTML element is used to create a cell in a table. The actual title in given in a hyperlink, encoded using `<a>`. The URL to the page for the movie is given in an 'href' attribute. 

There is one additional challenge that you need to be aware of. The IMDB website only responds to requests received from web scraping scripts if these requests also specify a 'User-Agent' in the a header. Each HTTP request contains a header, which provides important metadata about the request. The 'User-Agent' in this header typically gives imformation about the Computer and the browser from which the request was sent. 

The easiest way to find an approproate value for a 'User-Agent' is to go to [a website listing some common options](https://scrapfly.io/blog/how-to-avoid-web-scraping-blocking-headers/), and to select the case that applies. 

The information about the 'User-Agent' in the header must be provided in the `header` attribute of the `get()` method of `requests`, in the form of a dictionary.  

In [8]:
import requests

url = 'https://www.imdb.com/chart/top/'

## You can use the value below if you use Firefox on a Mac
## Adjust the value of user_agent if that is not the case.
user_agent = '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:98.0) Gecko/20100101 Firefox/98.0"'
headers={"User-Agent": user_agent}

response = requests.get( url , headers=headers)

print(response.status_code)

if response:
    response.encoding = 'utf-8'
    html_page = response.text 


200


In [9]:
movies = soup.find_all('td', {'class': 'titleColumn'} )

for m in movies:
    # Find links (a elements) within the cell",
    children = m.findChildren("a" , recursive=False)
    for c in children:
        movie_title = c.text
        url = c.get('href')
        ## This is an internal link, so we need to prepend the base url
        url = 'https://imdb.com' + url
        print( f'{movie_title}: {url}' )  

### Exercise 10.8.

The webpage below offers access to the complete work of the author H.P. Lovecraft. 

https://www.hplovecraft.com/writings/texts/

    
Write code in Python to find and print the URLs of all the texts that are listed. The links are all encoded in an element named &lt;a&gt;. The attribute `href` mentions the links, and the body of the &lt;a&gt; element mentions the title. List only the web pages that end in '.aspx'. 


In [10]:
from bs4 import BeautifulSoup
import requests
import re

base_url = "http://www.hplovecraft.com/writings/texts/"

response = requests.get(base_url)
if response: 
    #print(response.text)
    soup = BeautifulSoup( response.text ,"lxml")
    links = soup.find_all("a")
    for link in links:
        if link.get('href') is not None:
            title = link.string
            url = base_url + link.get('href')
            if re.search( r'aspx$' , url): 
                print( f'{title}\n{url}')

The Alchemist
http://www.hplovecraft.com/writings/texts/fiction/a.aspx
Ashes
http://www.hplovecraft.com/writings/texts/fiction/as.aspx
At the Mountains of Madness
http://www.hplovecraft.com/writings/texts/fiction/mm.aspx
Azathoth
http://www.hplovecraft.com/writings/texts/fiction/az.aspx
The Battle that Ended the Century
http://www.hplovecraft.com/writings/texts/fiction/bec.aspx
The Beast in the Cave
http://www.hplovecraft.com/writings/texts/fiction/bc.aspx
Beyond the Wall of Sleep
http://www.hplovecraft.com/writings/texts/fiction/bws.aspx
The Book
http://www.hplovecraft.com/writings/texts/fiction/b.aspx
The Call of Cthulhu
http://www.hplovecraft.com/writings/texts/fiction/cc.aspx
The Case of Charles Dexter Ward
http://www.hplovecraft.com/writings/texts/fiction/cdw.aspx
The Cats of Ulthar
http://www.hplovecraft.com/writings/texts/fiction/cu.aspx
Celephaïs
http://www.hplovecraft.com/writings/texts/fiction/c.aspx
The Challenge from Beyond
http://www.hplovecraft.com/writings/texts/fiction/

### Exercise 10.9.

Using `requests` and `BeautifulSoup`, create a list of all the countries mentioned on https://www.scrapethissite.com/pages/simple/.

Also collect and print data about the capital, the population and the area of all of these countries.

How you print or present the information is not too important here; the challenge in this exercise is to extract the data from the webpage.

In [11]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.scrapethissite.com/pages/simple/'

response = requests.get(url)

if response.status_code == 200:
    response.encoding = 'utf-8'
    html_page = response.text
    
    
soup = BeautifulSoup( html_page,"lxml")
    
countries = soup.find_all('div', {'class': 'col-md-4 country'} )


for c in countries:
    
    name = c.find('h3' , { 'class':'country-name'})
    print(name.text.strip())
    
    capital = c.find('span', { 'class':'country-capital'}).text
    population = c.find('span', { 'class':'country-population'}).text
    area = c.find('span', { 'class':'country-area'}).text
    
    print(f'  Capital: {capital}')
    print(f'  Population: {population}')
    print(f'  Area: {area}')
    print()

Andorra
  Capital: Andorra la Vella
  Population: 84000
  Area: 468.0

United Arab Emirates
  Capital: Abu Dhabi
  Population: 4975593
  Area: 82880.0

Afghanistan
  Capital: Kabul
  Population: 29121286
  Area: 647500.0

Antigua and Barbuda
  Capital: St. John's
  Population: 86754
  Area: 443.0

Anguilla
  Capital: The Valley
  Population: 13254
  Area: 102.0

Albania
  Capital: Tirana
  Population: 2986952
  Area: 28748.0

Armenia
  Capital: Yerevan
  Population: 2968000
  Area: 29800.0

Angola
  Capital: Luanda
  Population: 13068161
  Area: 1246700.0

Antarctica
  Capital: None
  Population: 0
  Area: 1.4E7

Argentina
  Capital: Buenos Aires
  Population: 41343201
  Area: 2766890.0

American Samoa
  Capital: Pago Pago
  Population: 57881
  Area: 199.0

Austria
  Capital: Vienna
  Population: 8205000
  Area: 83858.0

Australia
  Capital: Canberra
  Population: 21515754
  Area: 7686850.0

Aruba
  Capital: Oranjestad
  Population: 71566
  Area: 193.0

Åland
  Capital: Mariehamn
  Pop

### Exercise 10.10.

Download all the images shown on the following page: <https://www.bbc.com/news/in-pictures-61014501>

You can follow these steps:

1. Download the HTML file
1. 'Scrape' the HTML file you downloaded. As images in HTML are encoded using the `<img>` element, try to create a list containing all occurrences of this element. 
1. Find the URLS of all the images. Within these `<img>` element, there should be a `src` attribute containing the URL of the image. 
1. The bbc.com website uses images as part of the user interface. These images all have the word 'line' in their filenames. Try to exclude these images whose file names contain the word 'line'. 
1. Download all the images that you found in this way, using the `requests` library. In the `Response` object that is created following a succesful download, you need to work with the `content` property to obtain the actual file.  Save all these images on your computer, using `open()` and `write()`. In the `open()` function, use the string `"wb"` (write binary) as a second parameter (instead of only `"w"`) to make sure that the contents are saved as bytes.


In [12]:
import os

url = 'https://www.bbc.com/news/in-pictures-61014501'

response = requests.get(url)

if response:
    html_page = response.text
    soup = BeautifulSoup( html_page,"lxml")
    images = soup.find_all('img')
    for i in images:
        img_url = i.get('src')
        if 'line' not in img_url:
            response = requests.get(img_url)
            if response:
                file_name = os.path.basename(img_url)
                print(file_name)
                out = open( file_name , 'wb' )
                out.write(response.content)
                out.close()
    

_124077528_a46be0aa-415a-4763-8006-b062372f2902.jpg
_124077529_a74db2fe-7fe6-4c02-8718-63c130e948d0.jpg
_124077530_0f299f7f-93e2-4745-a9c8-6710c96bf7aa.jpg
_124082421_606cc114-7490-465b-973e-e5564898432d.jpg
_124077532_0ffb6354-8ad5-4775-9097-c8f49232254a.jpg
_124066216_015_world-press-photo-open-format-award_isadora-romero_poster.jpg
_124066217_020_af1.jpg
_124077533_933e6432-60a3-40fe-bab3-5e220f35ca94.jpg
_124077534_4d53719a-e81f-4f48-971a-780fadf8ea8f.jpg
_124077535_b136a62a-d259-41a3-ae6c-0811fde1b261.jpg
_124077536_17981911-334b-4d03-b1d2-0f5964cda90b.jpg
_124078317_04b32159-c34e-4b49-991d-5491481b08b8.jpg
_124078318_8e16ee76-73ae-463d-bcce-435c9680b5ad.jpg
_124074641_116_south-america_single_vladimir-encina.jpg
_124078320_810a2b8c-fc75-4cea-b872-ad78210ed34a.jpg
_124078319_e77bba76-be03-49ff-b698-c05a3612d8e4.jpg
_123131223_magdalena-wasiczek--374832.jpg
_122743256_35ab709a-5879-4793-86d8-fe6021a889b8.jpg
_122360774_c_gettyimages-1345162368.jpg


### Exercise 10.11.

Write Python code which can download the titles and the URLs of Wikipedia articles whose titles contain the word 'Dutch'. Your code needs to display the first 30 results only.

You can search for Wikipedia pages containing a certain term using the following base URL: 

```
base_url = 'https://en.wikipedia.org/w/api.php?action=opensearch'
```


As you can read in the [documentation of this API](https://www.mediawiki.org/w/api.php?action=help&modules=opensearch), the `opensearch` function accepts the following parameters: 

* *query* speficies the search term. 
* *limit* sets a limit to the number of items to return
* For the *format*, you can choose either 'xml' or 'json'. 

If you request data in the JSON format, and convert the data using the `json()` method of `requests`, these data will be structured in quite a particular way. At the first level, the object is list containing four items. The second item is another list, containing the titles of the articles. The fourth item is yet another list, containing the URLs of all of these articles. 

Note that this can be a challenging exercise! Don't feel discouraged if you cannot manage to compelete the exercise without looking at the solutions. 

In [13]:
import requests
import json

# Let's construct the full API call (which is a URL) piece by piece
base_url = 'https://en.wikipedia.org/w/api.php?action=opensearch'

search_term = "Dutch"
limit = 30
data_format = 'json'

api_call = f'{base_url}&search={search_term}&limit={limit}&format={data_format}'

# Get the data using the Requests library
response_data = requests.get( api_call )

# Because we asked for and got JSON-formatted data, Requests lets us access
# the data as a Python data structure using the .json() method
wiki_results = response_data.json()

# Now we print the search results 
for i in range( 0 , len(wiki_results[1]) ):
    print( 'Title: ' + wiki_results[1][i] )
    print( 'Tagline: ' + wiki_results[2][i] )
    print( 'Url: ' + wiki_results[3][i] + '\n')


Title: Dutch
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch

Title: Dutch language
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_language

Title: Dutch East India Company
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_East_India_Company

Title: Dutch people
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_people

Title: Dutch East Indies
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_East_Indies

Title: Dutch colonial empire
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_colonial_empire

Title: Dutch Golden Age
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_Golden_Age

Title: Dutchess County, New York
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutchess_County,_New_York

Title: Dutch cuisine
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_cuisine

Title: Dutch Formosa
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_Formosa

Title: Dutch Americans
Tagline: 
Url: https://en.wikipedia.org/wiki/Dutch_Americans

Title: Dutch Republic
Tagline: 
Url: https://